# Transformer、ELMO和GPT

Transformer、ELMO和GPT都是自然语言处理领域中的重要模型，它们在不同的任务和应用中发挥着关键作用。下面是它们之间的简要区别和联系：

1. Transformer（变形金刚）:
Transformer是一种用于序列到序列（Sequence-to-Sequence）任务的神经网络模型，最初被广泛应用于机器翻译任务。它引入了自注意力机制（Self-Attention）来建立输入序列中单词之间的关联性。Transformer的核心思想是通过多层的自注意力和前馈神经网络层来进行序列建模。它的优势在于可以并行计算，因此在处理长序列时更高效。

2. ELMO（Embeddings from Language Models）:
ELMO是一种基于上下文词向量（Contextual Word Embeddings）的表示学习模型。ELMO使用双向语言模型来生成每个单词的词向量，同时考虑上下文的语义信息。与传统的词向量模型（如Word2Vec和GloVe）不同，ELMO的词向量是根据整个句子的语境动态生成的。这使得ELMO能够更好地捕捉词汇的多义性和上下文相关性。

3. GPT（Generative Pre-trained Transformer）:
GPT是一种基于Transformer架构的预训练语言模型。GPT采用了无监督的方式进行预训练，使用大规模的语料库来学习语言的统计特征。预训练完成后，GPT可以用于多种下游任务，如文本生成、问答系统和语义理解。GPT的一个关键特点是使用了自回归（autoregressive）的方式，即在生成输出时，依次生成每个单词，利用先前生成的单词来指导后续生成。

联系：
- Transformer是GPT和ELMO的底层架构，两者都使用了Transformer模型。
- ELMO和GPT都是预训练模型，利用大规模语料进行训练，然后在下游任务上进行微调。
- GPT和ELMO都可以用于多个自然语言处理任务，如文本生成、文本分类和命名实体识别等。

区别：
- ELMO是一种基于上下文的词向量模型，重点关注每个单词在不同上下文中的语义表示。而GPT是一个预训练语言模型，它生成下一个单词的概率分布。
- GPT使用了自回归（autoregressive）的方式，即生成输出时依次生成每个单词，而ELMO并没有显式的生成过程。
- GPT生成的是一段连续的文本，而EL

MO生成的是每个单词的上下文向量。
- GPT通常用于生成文本和问答任务，而ELMO更适用于文本分类和句子级别的任务。

综上所述，Transformer是底层架构，ELMO是基于上下文的词向量模型，而GPT是一种预训练语言模型，它们在某些方面有共同之处，但也有一些重要的区别。这些模型各自有不同的优势和适用场景，根据具体任务的需求来选择合适的模型。

## ELMo

In [7]:
from torch import nn, optim
import torch
from torch.nn.functional import cross_entropy, softmax
import utils
from torch.utils.data import DataLoader
import os

In [8]:
class ELMo(nn.Module):
    def __init__(self, v_dim, emb_dim, units, n_layers, lr):
        super(ELMo, self).__init__()
        self.n_layers = n_layers
        self.units = units
        self.v_dim = v_dim

        #encoder
        self.word_embed = nn.Embedding(num_embeddings=v_dim, embedding_dim=emb_dim, padding_idx=0)
        self.word_embed.weight.data.normal_(0, 0.1)

        #forward LSTM
        self.fs = nn.ModuleList([
            nn.LSTM(input_size=emb_dim, hidden_size=units, batch_first=True)
            if i == 0 else
            nn.LSTM(input_size=units, hidden_size=units, batch_first=True)
            for i in range(n_layers)
        ])
        self.f_logits = nn.Linear(in_features=units, out_features=v_dim)

        #backward LSTM
        self.bs = nn.ModuleList([
            nn.LSTM(input_size=emb_dim, hidden_size=units, batch_first=True)
            if i == 0 else
            nn.LSTM(input_size=units, hidden_size=units, batch_first=True)
            for i in range(n_layers)
        ])
        self.b_logits = nn.Linear(in_features=units, out_features=v_dim)

        self.opt = optim.Adam(self.parameters(), lr=lr)

    def forward(self,seqs):
        device = next(self.parameters()).device
        embedded = self.word_embed(seqs)    # [n, step, emb_dim]
        fxs = [embedded[:, :-1, :]]         # [n, step-1, emb_dim]
        bxs = [embedded[:, 1:, :]]          # [n, step-1, emb_dim]
        (h_f,c_f) = (torch.zeros(1,seqs.shape[0],self.units).to(device),torch.zeros(1,seqs.shape[0],self.units).to(device)) #初始化前向传播的隐藏状态(h_f,c_f)为全零张量，这里使用了LSTM模型，因此h_f和c_f的维度为[1, n, units]，其中n表示批量大小，units表示LSTM隐藏单元的数量。
        (h_b,c_b) = (torch.zeros(1,seqs.shape[0],self.units).to(device),torch.zeros(1,seqs.shape[0],self.units).to(device)) #初始化后向传播的隐藏状态(h_b, c_b)为全零张量
        #对于前向和后向的LSTM层对，使用zip函数遍历前向层列表self.fs和后向层列表self.bs
        for fl,bl in zip(self.fs,self.bs):
            # 对于当前前向LSTM层fl，将前一个步骤的输出fxs[-1]和前向隐藏状态(h_f, c_f)作为输入，执行前向传播操作。返回前向传播的输出张量output_f和更新后的前向隐藏状态(h_f, c_f)
            output_f,(h_f,c_f) = fl(fxs[-1], (h_f,c_f))   # [n, step-1, units], [1, n, units]
            fxs.append(output_f)
            # 对于当前后向LSTM层bl，将翻转的前一个步骤的输出torch.flip(bxs[-1],dims=[1,])和后向隐藏状态(h_b, c_b)作为输入，执行后向传播操作。返回后向传播的输出张量output_b和更新后的后向隐藏状态(h_b, c_b)
            output_b,(h_b,c_b) = bl(torch.flip(bxs[-1],dims=[1,]), (h_b,c_b)) # [n, step-1, units], [1, n, units]
            bxs.append(torch.flip(output_b,dims=(1,)))
        return fxs,bxs

    def step(self,seqs):
        self.opt.zero_grad()
        fo,bo = self(seqs)  #调用forward函数，返回前向LSTM层和后向LSTM层的输出
        # 全连接层
        fo = self.f_logits(fo[-1])  # [n, step-1, v_dim]
        bo = self.b_logits(bo[-1])  # [n, step-1, v_dim]
        # 损失函数
        loss = (
            cross_entropy(fo.reshape(-1,self.v_dim),seqs[:,1:].reshape(-1)) +
            cross_entropy(bo.reshape(-1,self.v_dim),seqs[:,:-1].reshape(-1)))/2
        loss.backward()
        self.opt.step()
        return loss.cpu().detach().numpy(), (fo,bo)

    def get_emb(self,seqs):
        fxs,bxs = self(seqs)    #调用forward函数，返回前向LSTM层和后向LSTM层的输出
        xs = [
            #创建一个列表xs，将前向传播和后向传播的第一层的输出进行拼接。使用torch.cat函数将前向传播的输出fxs[0][:,1:,:]和后向传播的输出bxs[0][:,:-1,:]在维度2上进行拼接。然后将拼接后的张量转移到CPU上，并使用.data.numpy()将其转换为NumPy数组
            torch.cat((fxs[0][:,1:,:],bxs[0][:,:-1,:]),dim=2).cpu().data.numpy()
        ] + [
            #对于后续的前向传播和后向传播的输出层，使用列表推导式遍历fxs[1:]和bxs[1:]，分别表示从第二层开始的前向传播和后向传播的输出。对每一层，使用torch.cat函数将前向传播的输出f[:,1:,:]和后向传播的输出b[:,:-1,:]在维度2上进行拼接。然后将拼接后的张量转移到CPU上，并使用.data.numpy()将其转换为NumPy数组
            torch.cat((f[:,1:,:],b[:,:-1,:]),dim=2).cpu().data.numpy() for f,b in zip(fxs[1:],bxs[1:])
        ]
        for x in xs:    #遍历列表xs中的每个嵌入表示，并打印其形状信息
            print("layers shape=",x.shape)
        return xs   #返回嵌入表示列表xs

In [9]:
def export_w2v(model,data,device):
    # 加载训练好的ELMo模型的参数。通过torch.load函数加载模型参数文件model.pth，并使用map_location参数将模型加载到指定的设备上（CPU或GPU）。加载后的模型参数会更新到model对象中
    model.load_state_dict(torch.load("./visual/models/elmo/model.pth",map_location=device))
    emb = model.get_emb(data)   #调用模型的get_emb函数，传入输入数据data，获取输入数据的嵌入表示。将嵌入表示赋值给变量emb
    print(emb)

In [10]:
def train():
    dataset = utils.MRPCSingle("./MRPC",rows=2000)
    UNITS = 256
    N_LAYERS = 2
    BATCH_SIZE = 16
    LEARNING_RATE = 2e-3
    print('num word: ',dataset.num_word)
    model = ELMo(v_dim = dataset.num_word,emb_dim = UNITS, units=UNITS, n_layers=N_LAYERS,lr=LEARNING_RATE)
    if torch.cuda.is_available():
        print("GPU train avaliable")
        device =torch.device("cuda")
        model = model.cuda()
    else:
        device = torch.device("cpu")
        model = model.cpu()
    loader = DataLoader(dataset,batch_size=BATCH_SIZE,shuffle=True)
    for i in range(10):
        for batch_idx , batch in enumerate(loader):
            batch = batch.type(torch.LongTensor).to(device)
            loss, (fo,bo) = model.step(batch)
            if batch_idx % 20 ==0:
                fp = fo[0].cpu().data.numpy().argmax(axis=1)
                bp = bo[0].cpu().data.numpy().argmax(axis=1)
                print("\n\nEpoch: ", i,
                "| batch: ", batch_idx,
                "| loss: %.3f" % loss,
                "\n| tgt: ", " ".join([dataset.i2v[i] for i in batch[0].cpu().data.numpy() if i != dataset.pad_id]),
                "\n| f_prd: ", " ".join([dataset.i2v[i] for i in fp if i != dataset.pad_id]),
                "\n| b_prd: ", " ".join([dataset.i2v[i] for i in bp if i != dataset.pad_id]),
                )
    os.makedirs("./visual/models/elmo",exist_ok=True)
    # 使用torch.save函数将模型的状态字典（state_dict）保存为文件。状态字典包含了模型的所有参数和缓冲区，可以通过加载该文件来还原模型的状态。
    torch.save(model.state_dict(),"./visual/models/elmo/model.pth")
    # 调用export_w2v函数，传入模型对象model、数据batch[:4]和设备device。这行代码执行的是前面提到的导出模型词向量的过程
    export_w2v(model,batch[:4],device)

In [11]:
if __name__ == "__main__":
    train()

num word:  12880
GPU train avaliable


Epoch:  0 | batch:  0 | loss: 9.474 
| tgt:  <GO> the women then had follow-up examinations after five , <NUM> and <NUM> years . <SEP> 
| f_prd:  plain suem smaller seems smaller smaller smaller smaller smaller smaller smaller smaller smaller low-income smaller smaller smaller smaller smaller smaller smaller smaller smaller smaller smaller smaller smaller smaller smaller smaller smaller smaller smaller smaller smaller smaller smaller 
| b_prd:  relatively relatively relatively relatively relatively relatively relatively relatively relatively relatively relatively relatively relatively relatively relatively relatively relatively relatively relatively relatively relatively relatively relatively relatively relatively relatively relatively relatively relatively relatively relatively relatively relatively relatively hostages hostages petitioned


Epoch:  0 | batch:  20 | loss: 5.160 
| tgt:  <GO> the us government and private technology experts have wa

## GPT

In [1]:
from transformer import Encoder
from torch import nn,optim
from torch.nn.functional import cross_entropy,softmax, relu
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate

import torch
import utils
import os
import pickle

In [2]:
class GPT(nn.Module):
    def __init__(self,model_dim,max_len,num_layer,num_head,n_vocab,lr,
                 max_seg=3,drop_rate=0.2,padding_idx=0):
        super(GPT, self).__init__()
        self.padding_idx=padding_idx    #掩蔽位置的索引
        self.n_vocab=n_vocab    #词汇表大小
        self.max_len=max_len    #输入序列的最大长度

        self.word_emb=nn.Embedding(n_vocab,model_dim)   # 词嵌入
        self.word_emb.weight.data.normal_(0,0.1)    # 初始化词嵌入层的权重

        self.segment_emb=nn.Embedding(num_embeddings=max_seg,embedding_dim=model_dim)   #分段嵌入层，将最大分段数量和模型维度作为参数
        self.segment_emb.weight.data.normal_(0,0.1) #初始化分段嵌入层的权重
        self.position_emb=torch.empty(1,max_len,model_dim)  #位置嵌入
        nn.init.kaiming_normal_(self.position_emb,mode='fan_out',nonlinearity='relu')   #kaiming正太分布初始化
        self.position_emb=nn.Parameter(self.position_emb)   #将位置向量转换为可学习的参数，使其可通过反省传播进行优化

        self.encoder=Encoder(n_head=num_head,emb_dim=model_dim,drop_rate=drop_rate,n_layer=num_layer)   #编码器
        self.task_mlm=nn.Linear(in_features=model_dim,out_features=n_vocab)
        self.task_nsp=nn.Linear(in_features=model_dim*self.max_len,out_features=2)

        self.opt=optim.Adam(self.parameters(),lr)   #self.parameters()获取类中所有可训练参数
        # self.parameters() 方法只会返回直接属于该类的可训练参数，而不会递归地获取其子模块中的可训练参数。如果想要获取所有子模块中的可训练参数，可以使用 model.named_parameters() 方法。

    def forward(self,seqs,segs,training=False):
        embed=self.input_emb(seqs,segs) #获取输入序列和分段嵌入表示，它将词嵌入、分段嵌入和位置编码相加，形成输入的嵌入表示
        z=self.encoder(embed,training,mask=self.mask(seqs)) #将输入的嵌入表示传递给编码器，编码器对嵌入表示进行处理，同时根据输入序列生成由mask方法生成的掩码，返回编码后的表示z
        mlm_logits=self.task_mlm(z) #将编码后的表示 z 输入到 MLM（Masked Language Modeling）任务的线性层 self.task_mlm 中，得到用于预测被掩码标记的单词的 logits
        nsp_logits=self.task_nsp(z.reshape(z.shape[0],-1))  #将编码后的表示 z 重塑为二维张量，并将其输入到 NSP（Next Sentence Prediction）任务的线性层 self.task_nsp 中。它生成用于预测句子对关系的 logits
        return mlm_logits,nsp_logits

    def step(self, seqs, segs, seqs_, nsp_labels):
        self.opt.zero_grad()
        mlm_logits, nsp_logits = self(seqs, segs, training=True)
        pred_loss = cross_entropy(mlm_logits.reshape(-1,self.n_vocab),seqs_.reshape(-1))
        nsp_loss = cross_entropy(nsp_logits,nsp_labels.reshape(-1))
        loss = pred_loss + 0.2 * nsp_loss
        loss.backward()
        self.opt.step()
        return loss.cpu().data.numpy(), mlm_logits

    def input_emb(self,seqs, segs):
        # device = next(self.parameters()).device
        # self.position_emb = self.position_emb.to(device)
        return self.word_emb(seqs) + self.segment_emb(segs) + self.position_emb

    def mask(self, seqs):
        device = next(self.parameters()).device
        batch_size, seq_len = seqs.shape
        mask = torch.triu(torch.ones((seq_len,seq_len), dtype=torch.long), diagonal=1).to(device)  # [seq_len ,seq_len]
        pad = torch.eq(seqs,self.padding_idx)   # [n, seq_len]
        mask = torch.where(pad[:,None,None,:],1,mask[None,None,:,:]).to(device)   # [n, 1, seq_len, seq_len]
        return mask>0   # [n, 1, seq_len, seq_len]

    @property
    def attentions(self):
        attentions = {
            "encoder": [l.mh.attention.cpu().data.numpy() for l in self.encoder.encoder_layers]
        }
        return attentions

# nn.Parameter()和self.segment_emb.weight.data.normal_(0,0.1)

这两个方法分别是PyTorch中对模型参数初始化的不同方式。

首先，`self.position_emb=nn.Parameter(self.position_emb)` 是将一个Tensor类型的变量`self.position_emb`转换成为可训练的模型参数。在神经网络中，模型参数通常需要被学习来最小化损失函数，而 `nn.Parameter()` 可以将一个Tensor类型的变量添加到模型参数中，并且可以被自动优化器更新。

另一方面，`self.segment_emb.weight.data.normal_(0,0.1)` 基于正态分布随机初始化了一个权重矩阵。这里的 `normal_()` 方法指定了均值 0 和标准差 0.1 的正态分布来初始化权重矩阵。

总的来说，`self.position_emb=nn.Parameter(self.position_emb)` 将已有的Tensor类型变量转换成为可训练的模型参数；而 `self.segment_emb.weight.data.normal_(0,0.1)` 则是用指定分布随机初始化新的模型参数。

# nn.init.kaiming_normal_()

这行命令使用了 PyTorch 中的 `nn.init.kaiming_normal_()` 函数，对模型中的 `self.position_emb` 参数进行了初始化，具体作用如下：

- `nn.init.kaiming_normal_()` 是一种参数初始化方法，它使用了 Kaiming He 等人提出的一种针对 ReLU 激活函数的初始化方法，可以更好地保证神经网络的稳定性和收敛速度。
- 在这里，`mode='fan_out'` 表示使用输出通道数来计算权重的标准差，`nonlinearity='relu'` 表示使用 ReLU 作为激活函数。因此，这个初始化方法会根据输出通道数和激活函数类型自动计算标准差，并随机生成符合高斯分布的参数值，用于初始化 `self.position_emb` 这个张量。
- `self.position_emb` 可能是一个位置嵌入矩阵，用于在 Transformer 等模型中编码序列中每个位置的信息。通过使用 `nn.init.kaiming_normal_()` 进行初始化，可以使得这些位置嵌入的值具有一定的随机性，从而增加了模型的表达能力和泛化能力。

总之，这行命令的作用是使用 Kaiming He 的初始化方法，对模型中的位置嵌入参数进行初始化，以提高模型的稳定性和表达能力。

# 分段嵌入层

分段嵌入层（Segment Embedding Layer）是用于对输入序列进行分段信息编码的一种嵌入层。在某些自然语言处理任务中，除了词语本身的语义信息外，分段信息也可以对模型的表征能力和性能有所帮助。例如，在机器翻译任务中，分段信息可以指示源语言和目标语言的边界，帮助模型理解句子的结构。

词嵌入层（Word Embedding Layer）则是将词语映射为连续的向量表示的层。它通过学习词语在一个低维空间中的表示，捕捉了词语之间的语义关系，提供了一种有效的方式来表示词语的语义信息。

分段嵌入层和词嵌入层在模型中的作用有所不同：

1. 词嵌入层：将词语映射为连续向量表示，用于表示词语的语义信息。

2. 分段嵌入层：用于编码输入序列的分段信息，提供关于序列中各个部分之间边界的信息。

在模型的输入阶段，通常将输入序列进行分段，并为每个分段分配一个对应的标识符。词嵌入层用于为每个词语生成向量表示，而分段嵌入层用于为每个分段生成向量表示。在模型的后续层中，词嵌入和分段嵌入的向量表示可以通过加和或拼接等方式进行融合，以综合利用词语的语义信息和分段的结构信息。

综上所述，分段嵌入层和词嵌入层在模型中具有不同的作用，但它们共同为模型提供了对输入序列的丰富表示，从而有助于模型理解和处理自然语言的语义和结构信息。

分段嵌入层的分段依据可以根据具体任务和数据集的需求而定。常见的分段依据包括句子级别的分段和标记级别的分段。

1. 句子级别的分段：在某些任务中，例如文本分类、情感分析等，句子是任务的基本处理单位。在这种情况下，可以将每个句子作为一个分段，并为每个句子分配一个对应的标识符。这样，分段嵌入层可以为每个句子生成一个独立的向量表示，以帮助模型理解句子之间的关系和顺序。

2. 标记级别的分段：在其他任务中，可能需要更细粒度的分段信息，例如机器翻译、文本生成等。在这种情况下，可以根据标点符号或其他特定的分隔符将输入序列划分为多个分段，并为每个分段分配一个对应的标识符。这样，分段嵌入层可以为每个分段生成一个向量表示，帮助模型理解分段之间的边界和关系。

分段嵌入层与句向量有一些区别：

- 分段嵌入层：为输入序列的每个分段生成嵌入向量表示，关注序列的结构和分段之间的关系。它提供了关于序列中各个部分之间边界的信息，帮助模型理解和处理序列的结构。

- 句向量：句向量是对整个句子进行编码得到的一个向量表示，通常通过对句子中的词语进行编码汇聚而得到。句向量旨在捕捉整个句子的语义信息，它可以作为句子的紧凑表示，用于下游任务的输入。

分段嵌入层和句向量的区别在于其关注点和作用范围不同。分段嵌入层更关注输入序列的结构和分段之间的关系，用于为每个分段生成向量表示；而句向量更关注整个句子的语义信息，用于表示整个句子的特征。它们在模型中的使用方式和应用场景也有所不同。

# MLM和NSP

当然，我可以详细介绍一下MLM（Masked Language Modeling）和NSP（Next Sentence Prediction）任务。

1. MLM（Masked Language Modeling）任务：
   MLM任务是一种基于掩码的语言建模任务，旨在预测输入文本中被掩码标记的单词。在GPT模型中，MLM任务通过将部分输入序列中的单词随机掩码或替换为特殊的掩码标记，然后要求模型预测这些被掩码的单词。模型通过学习上下文信息和语言规律来进行预测。

   在GPT类中，MLM任务由一个线性层 `self.task_mlm` 来实现。该线性层的输入是经过编码器处理后的表示 `z`，输出是对词汇表中各个单词的logits。通过对logits进行softmax操作，可以获得每个单词的预测概率分布。训练时，通过与真实标签计算交叉熵损失来优化模型参数，使得模型能够更准确地预测被掩码的单词。

2. NSP（Next Sentence Prediction）任务：
   NSP任务是用于判断两个句子是否是连续语义的任务。在GPT模型中，NSP任务通过将一对句子输入模型，并判断它们是否是连续语义关系。这对句子可以是相邻的句子，也可以是随机选择的句子对。模型通过学习两个句子之间的语义关系，进一步提升对上下文的理解能力。

   在GPT类中，NSP任务由一个线性层 `self.task_nsp` 来实现。该线性层的输入是经过编码器处理后的表示 `z`，经过了形状重塑以适应任务的需要。输出是判断两个句子之间语义关系的logits。通过对logits进行softmax操作，可以获得两个句子之间是连续语义还是不连续语义的预测概率分布。训练时，通过与真实标签计算交叉熵损失来优化模型参数，使得模型能够更准确地预测句子对的语义关系。

MLM和NSP任务在GPT模型中是为了提升语言模型的能力，使其能够更好地理解和生成文本。这些任务的设计是为了引入对上下文和语义的建模，使模型能够更好地学习语言的规律和语义信息。通过同时进行MLM和NSP任务的训练，GPT模型可以获得更强大的语言表示能力。

In [4]:
def export_attention(model,device,data,name="gpt"):
    """将模型的注意力矩阵和输入序列保存到文件中，以供后续分析和可视化使用。注意力矩阵可以用来观察模型在输入序列中的注意力分布情况，以及模型对不同位置的关注程度"""

    # 加载预训练的模型参数，使用torch.load函数加载模型参数，并通过map_location参数指定设备
    model.load_state_dict(torch.load("./visual/models/gpt/model.pth",map_location=device))
    # 从data中获取输入序列seqs、分段序列segs、序列长度xlen和NSP标签nsp_labels
    seqs, segs,xlen,nsp_labels = data[:32]
    # 将获取的数据转换为torch.Tensor类型，并存储在变量seqs、segs、xlen和nsp_labels中
    seqs, segs,xlen,nsp_labels = torch.from_numpy(seqs),torch.from_numpy(segs),torch.from_numpy(xlen),torch.from_numpy(nsp_labels)
    seqs, segs,nsp_labels = seqs.type(torch.LongTensor).to(device), segs.type(torch.LongTensor).to(device),nsp_labels.to(device)
    # 调用模型对象model，传递seqs和segs作为输入，并设置training=False，即不进行训练。此步骤的目的是计算注意力矩阵，以便后续导出
    model(seqs[:,:-1],segs[:,:-1],False)
    seqs = seqs.cpu().data.numpy()  #将seqs从GPU设备移动到CPU设备，并将其转换为NumPy数组
    # 创建一个字典data，包含两个键值对：
    # 键为"src"，对应的值是一个嵌套列表，其中每个内部列表表示一个输入序列的词汇形式，通过查找字典data.i2v将seqs中的词汇索引转换为词汇形式。
    # 键为"attentions"，对应的值是模型对象model中的注意力矩阵，通过访问模型的attentions属性获取
    data = {"src": [[data.i2v[i] for i in seqs[j]] for j in range(len(seqs))], "attentions": model.attentions}
    path = "./visual/tmp/%s_attention_matrix.pkl" % name
    os.makedirs(os.path.dirname(path), exist_ok=True)
    # 使用pickle.dump函数将字典data保存到文件中，文件以二进制模式写入（"wb"）
    with open(path, "wb") as f:
        pickle.dump(data, f)

# 计算注意力

`调用模型对象model，传递seqs和segs作为输入，并设置training=False，即不进行训练。此步骤的目的是计算注意力矩阵，以便后续导出`
这个不是很理解，为什么不进行训练就可以计算注意力矩阵？

在Transformer模型中，注意力机制是通过对输入序列进行自注意力计算得到的。自注意力机制的计算过程不需要进行训练，它仅依赖于输入序列和模型的参数。因此，在推理或导出阶段，我们可以使用不进行训练的模型来计算注意力矩阵。

具体来说，当设置`training=False`时，模型在前向传播过程中不会应用任何与训练相关的操作，如Dropout或参数更新。相反，它将专注于计算输入序列的表示，包括注意力矩阵。通过使用不进行训练的模型进行前向传播，我们可以获取模型对输入序列中不同位置的关注程度，从而得到注意力矩阵。

这样做的目的是为了在推理或导出阶段分析模型的行为，并检查模型对输入序列的处理方式，例如确定模型关注的重点、观察注意力权重分布等。因此，在这种情况下，计算注意力矩阵是一种探索模型内部运行机制的方法，而不是训练模型。

In [5]:
def train():
    MODEL_DIM = 256
    N_LAYER = 4
    LEARNING_RATE = 1e-4
    dataset = utils.MRPCData("./MRPC",2000)
    print("num word: ",dataset.num_word)
    model = GPT(
        model_dim=MODEL_DIM, max_len=dataset.max_len-1, num_layer=N_LAYER, num_head=4, n_vocab=dataset.num_word,
        lr=LEARNING_RATE, max_seg=dataset.num_seg, drop_rate=0.2, padding_idx=dataset.pad_id
    )
    if torch.cuda.is_available():
        print("GPU train avaliable")
        device =torch.device("cuda")
        model = model.cuda()
    else:
        device = torch.device("cpu")
        model = model.cpu()

    loader = DataLoader(dataset,batch_size=32,shuffle=True)

    for epoch in range(100):
        for batch_idx, batch in enumerate(loader):
            seqs, segs,xlen,nsp_labels = batch
            seqs, segs,nsp_labels = seqs.type(torch.LongTensor).to(device), segs.type(torch.LongTensor).to(device),nsp_labels.to(device)
            # pred: [n, step, n_vocab]
            loss,pred = model.step(seqs=seqs[:,:-1], segs= segs[:,:-1], seqs_=seqs[:,1:], nsp_labels=nsp_labels)
            if batch_idx %100 == 0:
                pred = pred[0].cpu().data.numpy().argmax(axis = 1) # [step]
                print(
                    "Epoch: ",epoch,
                "|batch: ", batch_idx,
                "| loss: %.3f" % loss,
                "\n| tgt: ", " ".join([dataset.i2v[i] for i in seqs[0, 1:].cpu().data.numpy()[:xlen[0].sum()+1]]),
                "\n| prd: ", " ".join([dataset.i2v[i] for i in pred[:xlen[0].sum()+1]]),
                )
    os.makedirs("./visual/models/gpt",exist_ok=True)
    torch.save(model.state_dict(),"./visual/models/gpt/model.pth")
    export_attention(model,device,dataset)

In [6]:
if __name__ == "__main__":
    train()

downloading from https://mofanpy.com/static/files/MRPC/msr_paraphrase_train.txt
completed
downloading from https://mofanpy.com/static/files/MRPC/msr_paraphrase_test.txt
completed
num word:  12880
GPU train avaliable
Epoch:  0 |batch:  0 | loss: 9.727 
| tgt:  <quote> it still remains to be seen whether the revenue recovery will be short-lived or long-lived , <quote> mr. sprayregen said . <SEP> <quote> it remains to be seen whether the revenue recovery will be short- or long-lived , <quote> said james sprayregen , ual bankruptcy attorney , in court . 
| prd:  absentia 4,000-pound raging per bloodsworth provincial martha sorts 108,000-mph hewlett-packard daughter-in-law raging raging emily marriott sweet scorched asda forecasters exhausted machinery lucy smart technology-laced cover adc streets sbc returning warfare subject underwriters non-essential menopause seconds vicki killers front same-store estate ukrainian hcl somalia antonio mckinlay underwriters ritter slab resumes hayes honor